In [ ]:
#! pip install tqdm multiprocess

In [ ]:
import numpy as np
import pandas as pd
from multiprocess import Pool
import time 

from functions import evaluate_ar, evaluate_multiplier_iid, evaluate_ma
from bstrapping.synthetic_time_series.moving_average import MovingAverage
from bstrapping.bootstrap_procedures.weighted_bootstrap import WeightedBootstrap
from bstrapping.weights.auto_regressive_weights import AutoRegressiveWeights
from bstrapping.weights.moving_average import MovingAverageWeights
from bstrapping.weights.gaussian_weights import GaussianWeights

In [ ]:
mean = 4  # mean of the time series
alpha = 0.1

# Names of the stochastic processes

# Dependence coefficients of the stochastic processes, i.e. of the moving average processes
dependence_coefficient = np.array([0.5 ** i for i in range(1, 3)])

list_name_weights = ['AR',
                     'Multiplier',
                     'MA',
                     ]

In [ ]:
runs = 2

## Benchmark bootstraps

In [ ]:
sample_sizes = [1000,2000,5000,10000]

In [ ]:
%%time
%%capture

p = Pool()
durations_ar_mean = []
durations_ar_std = []
for sample_size in sample_sizes:
    duration_ar = []
    sample = MovingAverage(mean=mean, 
                           parameters=dependence_coefficient).generate_samples(sample_size)
    for _ in range(runs):
        start_time = time.perf_counter()
        WeightedBootstrap(samples=sample,
                          weights=AutoRegressiveWeights(samples=sample),
                          number_bootstrap_samples=20)
        end_time = time.perf_counter()
        duration_ar.append(end_time - start_time)
    
    durations_ar_mean.append(np.mean(duration_ar))
    durations_ar_std.append(np.std(duration_ar))


In [ ]:
%%time
%%capture

p = Pool()
durations_iid_mean = []
durations_iid_std = []
for sample_size in sample_sizes:
    duration_iid = []
    sample = MovingAverage(mean=mean, 
                           parameters=dependence_coefficient).generate_samples(sample_size)
    for _ in range(runs):
        start_time = time.perf_counter()
        WeightedBootstrap(samples=sample,
                          weights=GaussianWeights(samples=sample),
                          number_bootstrap_samples=20)
        end_time = time.perf_counter()
        duration_iid.append(end_time - start_time)
    
    durations_iid_mean.append(np.mean(duration_iid))
    durations_iid_std.append(np.std(duration_iid))

In [ ]:
%%time
%%capture

p = Pool()
durations_ma_mean = []
durations_ma_std = []
for sample_size in sample_sizes:
    duration_ma = []
    sample = MovingAverage(mean=mean, 
                           parameters=dependence_coefficient).generate_samples(sample_size)
    for _ in range(runs):
        start_time = time.perf_counter()
        WeightedBootstrap(samples=sample,
                          weights=MovingAverageWeights(samples=sample),
                          number_bootstrap_samples=20)
        end_time = time.perf_counter()
        duration_ma.append(end_time - start_time)
    
    durations_ma_mean.append(np.mean(duration_ma))
    durations_ma_std.append(np.std(duration_ma))

## Concatination result

In [ ]:
benchmark = pd.DataFrame(
    [durations_ar_mean,durations_iid_mean,durations_ma_mean,
    durations_ar_std,durations_iid_std,durations_ma_std],
    columns=sample_sizes,
    index= pd.MultiIndex.from_tuples([("mean","AR"),("mean","Multiplier"),("mean","MA"),
                                     ("std","AR"),("std","Multiplier"),("std","MA")])
).T

In [ ]:
benchmark

In [ ]:
#benchmark.to_csv(f"./data/benchmark_times.csv")
#benchmark.to_pickle(f"./data/benchmark_times.pkl")